## Características para modelos de aprendizaje automatico para comunidades

In [1]:
import pandas as pd
import os
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords



In [2]:
directorio_actual = os.getcwd()
print("El directorio de trabajo actual es:", directorio_actual)

El directorio de trabajo actual es: g:\Unidades compartidas\Rocío doctorado\Proyecto NLP\Códigos\Comunidades


In [3]:
comunidades = pd.read_csv('G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/comunidades.csv')
comunidades.shape

(503340, 12)

In [4]:
comunidades.columns

Index(['Unnamed: 0', 'retweet_count', 'full_text', 'posicion', 'user_name',
       'user_id', 'mencion', 'retweet', 'user_name_mencion', 'comunidad',
       'degree_centrality', 'postura'],
      dtype='object')

In [5]:
comunidades_filtradas=comunidades[['full_text', 'postura', 'retweet_count']]
comunidades_filtradas.head()

,full_text,postura,retweet_count
0,RT @PTSarg: Contamos con una ventaja que ellos...,1,14
1,En el fondo lo que les jode es nuestra liberta...,1,0
2,RT @femialborto: A vos que decís #Salvemoslasd...,1,308
3,RT @CampAbortoLegal: Diputadxs: ¿Qué significa...,1,1369
4,RT @CarrioJovenes: Que no coincidamos en el te...,0,16


Limpieza de los tweets

In [6]:
import re
import pandas as pd

# Función para limpiar los tweets
def clean_tweet(text):
    # Eliminar 'RT' al inicio del texto
    text = re.sub(r'^RT\s+', '', text)

    # Eliminar menciones (@usuario) y el ':' que pueda seguir
    text = re.sub(r'@\w+:\s*', '', text)

    # Eliminar cualquier otra mención sin el ':' y espacios innecesarios
    text = re.sub(r'@\w+', '', text)

    # Eliminar espacios extra generados por las eliminaciones
    text = text.strip()

    return text

# Aplicar la función de limpieza al DataFrame
comunidades_filtradas.loc[:, 'full_text_cleaned'] = comunidades_filtradas['full_text'].apply(clean_tweet)

# Crear nueva columna 'Question Mark': Si hay al menos un signo de interrogación (¿ o ?)
comunidades_filtradas.loc[:, 'Question Mark'] = comunidades_filtradas['full_text'].apply(lambda x: '?' in x or '¿' in x)

# Crear nueva columna 'Question Mark Count': Contar cuántos signos de interrogación hay (¿ o ?)
comunidades_filtradas.loc[:, 'Question Mark Count'] = comunidades_filtradas['full_text'].apply(lambda x: x.count('?') + x.count('¿'))

# Crear nueva columna 'Starts with Question Mark': Si el tweet empieza con al menos un signo de interrogación
comunidades_filtradas.loc[:, 'Starts with Question Mark'] = comunidades_filtradas['full_text'].apply(lambda x: x.strip().startswith('¿'))

# Crear nueva columna 'Ends with Mark Count': Si el tweet termina con al menos un signo de interrogación
comunidades_filtradas.loc[:, 'Ends with Mark Count'] = comunidades_filtradas['full_text'].apply(lambda x: x.strip().endswith('?'))

# Crear nueva columna 'Upper Ratio': Ratio de letras mayúsculas vs letras totales
comunidades_filtradas.loc[:, 'Upper Ratio'] = comunidades_filtradas['full_text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

# Crear nueva columna 'Exclamation Mark': Si hay al menos un signo de admiración
comunidades_filtradas.loc[:, 'Exclamation Mark'] = comunidades_filtradas['full_text'].apply(lambda x: '!' in x or '¡' in x)

# Crear nueva columna 'Exclamation Mark Count': Contar cuántos signos de admiración hay
comunidades_filtradas.loc[:, 'Exclamation Mark Count'] = comunidades_filtradas['full_text'].apply(lambda x: x.count('!') + x.count('¡'))

# Crear nueva columna 'Ellipsis Occurrence': Si hay 3 o más puntos seguidos
comunidades_filtradas.loc[:, 'Ellipsis Occurrence'] = comunidades_filtradas['full_text'].apply(lambda x: '...' in x)

# Crear nueva columna 'Hashtag Presence': Presencia de hashtags
comunidades_filtradas.loc[:, 'Hashtag Presence'] = comunidades_filtradas['full_text'].apply(lambda x: bool(re.search(r'#\w+', x)))

# Crear nueva columna 'Text Length': Largo del texto sin hashtags, URLs, ni menciones de cuentas de Twitter
def limpiar_texto(texto):
    texto_sin_hashtags_urls = re.sub(r'(@\w+|http\S+|#\w+)', '', texto)  # Remueve hashtags, URLs y menciones
    return len(texto_sin_hashtags_urls.strip())

comunidades_filtradas.loc[:, 'Text Length'] = comunidades_filtradas['full_text'].apply(limpiar_texto)

# Crear nueva columna 'URL Count': Cantidad de URLs en el texto
comunidades_filtradas.loc[:, 'URL Count'] = comunidades_filtradas['full_text'].apply(lambda x: len(re.findall(r'http\S+', x)))

# Crear nueva columna 'Quotes': Si hay palabras entre comillas
comunidades_filtradas.loc[:, 'Quotes'] = comunidades_filtradas['full_text'].apply(lambda x: bool(re.search(r'["“”]', x)))

comunidades_filtradas


C:\Users\Usuario\AppData\Local\Temp\ipykernel_20240\2013827799.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comunidades_filtradas.loc[:, 'full_text_cleaned'] = comunidades_filtradas['full_text'].apply(clean_tweet)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20240\2013827799.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comunidades_filtradas.loc[:, 'Question Mark'] = comunidades_filtradas['full_text'].apply(lambda x: '?' in x or '¿' in x)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20240\2

,full_text,postura,retweet_count,full_text_cleaned,Question Mark,Question Mark Count,Starts with Question Mark,Ends with Mark Count,Upper Ratio,Exclamation Mark,Exclamation Mark Count,Ellipsis Occurrence,Hashtag Presence,Text Length,URL Count,Quotes
0,RT @PTSarg: Contamos con una ventaja que ellos...,1,14,Contamos con una ventaja que ellos no tienen: ...,False,0,False,False,0.071429,False,0,False,True,116,0,False
1,En el fondo lo que les jode es nuestra liberta...,1,0,En el fondo lo que les jode es nuestra liberta...,False,0,False,False,0.114943,False,0,False,True,47,0,False
2,RT @femialborto: A vos que decís #Salvemoslasd...,1,308,A vos que decís #Salvemoslasdosvidas te pido q...,False,0,False,False,0.028571,False,0,False,True,108,0,False
3,RT @CampAbortoLegal: Diputadxs: ¿Qué significa...,1,1369,Diputadxs: ¿Qué significa para ustedes una muj...,True,2,False,False,0.135714,False,0,False,True,71,0,False
4,RT @CarrioJovenes: Que no coincidamos en el te...,0,16,Que no coincidamos en el tema del aborto con ...,False,0,False,False,0.050360,False,0,False,False,113,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503335,RT @NoAlAborto19: @danielscioli me bloqueo por...,0,10,me bloqueo porque le duele la verdad. En sus c...,False,0,False,False,0.042857,False,0,False,False,114,0,False
503336,RT @unidadprovida: “Hoy nuestra deuda es con l...,0,36,“Hoy nuestra deuda es con la mujeres más pobre...,False,0,False,False,0.021429,False,0,False,False,126,0,True
503337,"RT @lasmarias1234: #ArgentinaEsProvida"" #NoAlA...",0,115,"#ArgentinaEsProvida"" #NoAlAbortoEnArgentina #S...",False,0,False,False,0.107143,False,0,False,True,67,0,True
503338,RT @marianoobarrio: Esta inmensa maravilla fue...,0,174,Esta inmensa maravilla fue sacrificada hoy en ...,False,0,False,False,0.021583,False,0,False,False,124,0,False


# Eliminamos filas duplicadas

In [7]:
# Eliminar duplicados basándote en la columna 'full_text_cleaned' y conservar el primer tweet que aparezca
comunidades_filtradas_sin_duplicados = comunidades_filtradas.drop_duplicates(subset='full_text_cleaned', keep='first')

# Obtener los índices de las filas sin duplicados
indices_sin_duplicados = comunidades_filtradas_sin_duplicados.index
len(indices_sin_duplicados)

110409

In [8]:
# Filtrar comunidades_filtradas usando los índices sin duplicados
comunidades_filtradas_filtrada = comunidades_filtradas.loc[indices_sin_duplicados]

comunidades_filtradas_filtrada

,full_text,postura,retweet_count,full_text_cleaned,Question Mark,Question Mark Count,Starts with Question Mark,Ends with Mark Count,Upper Ratio,Exclamation Mark,Exclamation Mark Count,Ellipsis Occurrence,Hashtag Presence,Text Length,URL Count,Quotes
0,RT @PTSarg: Contamos con una ventaja que ellos...,1,14,Contamos con una ventaja que ellos no tienen: ...,False,0,False,False,0.071429,False,0,False,True,116,0,False
1,En el fondo lo que les jode es nuestra liberta...,1,0,En el fondo lo que les jode es nuestra liberta...,False,0,False,False,0.114943,False,0,False,True,47,0,False
2,RT @femialborto: A vos que decís #Salvemoslasd...,1,308,A vos que decís #Salvemoslasdosvidas te pido q...,False,0,False,False,0.028571,False,0,False,True,108,0,False
3,RT @CampAbortoLegal: Diputadxs: ¿Qué significa...,1,1369,Diputadxs: ¿Qué significa para ustedes una muj...,True,2,False,False,0.135714,False,0,False,True,71,0,False
4,RT @CarrioJovenes: Que no coincidamos en el te...,0,16,Que no coincidamos en el tema del aborto con ...,False,0,False,False,0.050360,False,0,False,False,113,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503323,@clarincom Y una vida terminada #NoAlAborto,0,0,Y una vida terminada #NoAlAborto,False,0,False,False,0.093023,False,0,False,True,20,0,False
503328,Ojalá los médicos se organicen y marchen para ...,0,0,Ojalá los médicos se organicen y marchen para ...,False,0,False,False,0.046875,False,0,False,True,116,0,False
503333,Debe estar senil ya pobre. Avísenle que el bla...,0,0,Debe estar senil ya pobre. Avísenle que el bla...,False,0,False,False,0.109848,False,0,False,True,97,1,False
503334,RT @todavidavale: La Academia Nacional de Dere...,0,2,La Academia Nacional de Derecho advierte que l...,False,0,False,False,0.071429,False,0,False,True,99,0,False


In [9]:
comunidades_filtradas_filtrada.columns

Index(['full_text', 'postura', 'retweet_count', 'full_text_cleaned',
       'Question Mark', 'Question Mark Count', 'Starts with Question Mark',
       'Ends with Mark Count', 'Upper Ratio', 'Exclamation Mark',
       'Exclamation Mark Count', 'Ellipsis Occurrence', 'Hashtag Presence',
       'Text Length', 'URL Count', 'Quotes'],
      dtype='object')

In [10]:
comunidades_variables_contextuales=comunidades_filtradas_filtrada.drop(['full_text', 'retweet_count', 'full_text_cleaned'], axis=1)
comunidades_variables_contextuales.columns

Index(['postura', 'Question Mark', 'Question Mark Count',
       'Starts with Question Mark', 'Ends with Mark Count', 'Upper Ratio',
       'Exclamation Mark', 'Exclamation Mark Count', 'Ellipsis Occurrence',
       'Hashtag Presence', 'Text Length', 'URL Count', 'Quotes'],
      dtype='object')

In [77]:

comunidades_variables_contextuales.to_csv('G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/Variables_contextuales_comunidades.csv', index=False)

## Obtencion de unigramas

Preprocesamiento de los textos para utilizarlos como entrada en los modelos de aprendizaje automático

In [11]:
# Asegúrate de tener el paquete de stopwords en español
nltk.download('stopwords')
df_preproces=pd.DataFrame()
# Inicializar el stemmer
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

def clean_tweet(tweet):
    # Eliminar 'RT' al inicio del texto (opcional, si quieres eliminar retweets)
    # tweet = re.sub(r'^RT\s+', '', tweet)

    # Eliminar menciones (@usuario), URLs y caracteres no alfabéticos
    tweet = re.sub(r'@\w+|http\S+', '', tweet)  # Eliminar menciones y URLs

    # Reemplazar '#' con una cadena vacía para conservar las palabras de los hashtags
    tweet = re.sub(r'#(\w+)', r'\1', tweet)  # Eliminar solo el símbolo '#'

    # Tokenizar el tweet y aplicar el stemmer
    tokens = tweet.split()
    tokens = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]

    return ' '.join(tokens)

# Suponiendo que tienes un DataFrame 'df' con una columna 'full_text_cleaned' y una columna 'etiqueta' (la clase)
df_preproces['cleaned_tweet'] = comunidades_filtradas_filtrada['full_text_cleaned'].apply(clean_tweet)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df_preproces

,cleaned_tweet
0,cont ventaj conviccion fuerz
1,en fond jod libert novotencontralasmujer abort...
2,a vos dec salvemoslasdosv pid escuch intent ab...
3,signif usted muj novotencontralasmujer abortol...
4,que coincid tem abort cambi respet admir
...,...
503323,y vid termin noalabort
503328,ojal medic organic march grit cuent los vam mi...
503333,deb senil avisenl blanc verd noalabort noalabo...
503334,la academi nacional derech adviert legaliz abo...


In [13]:
df_preproces['cleaned_tweet']

0                              cont ventaj conviccion fuerz
1         en fond jod libert novotencontralasmujer abort...
2         a vos dec salvemoslasdosv pid escuch intent ab...
3         signif usted muj novotencontralasmujer abortol...
4                  que coincid tem abort cambi respet admir
                                ...                        
503323                               y vid termin noalabort
503328    ojal medic organic march grit cuent los vam mi...
503333    deb senil avisenl blanc verd noalabort noalabo...
503334    la academi nacional derech adviert legaliz abo...
503339         gaston pierd tiemp hay q concentr q salg ley
Name: cleaned_tweet, Length: 110409, dtype: object

In [14]:
df_preproces['cleaned_tweet']
df_preproces['postura']=comunidades_filtradas_filtrada['postura']

In [15]:
df_preproces.columns

Index(['cleaned_tweet', 'postura'], dtype='object')

In [16]:
df_preproces['postura'].value_counts()

postura
1    65155
0    45254
Name: count, dtype: int64

Modelo de clasificación: el texto se convierte en vectores numéricos usando TF-IDF, y luego se entrena una regresión logística para predecir a qué clase pertenece cada tweet

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2


# Separar las características y las etiquetas
X = df_preproces['cleaned_tweet']
y = df_preproces['postura']  # Asegúrate de que esta columna contiene las clases

# Dividir el dataset en conjunto de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el pipeline para vectorización y regresión logística
tfidf_vectorizer = TfidfVectorizer()
X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)
X_test_vectorized = tfidf_vectorizer.transform(X_test)

# Entrenar el modelo de regresión logística
logistic_model = LogisticRegression(multi_class='ovr', max_iter=1000)
logistic_model.fit(X_train_vectorized, y_train)

# Evaluar el modelo en el conjunto de prueba
accuracy_test = logistic_model.score(X_test_vectorized, y_test)
print(f'Accuracy en el conjunto de prueba: {accuracy_test:.2f}')

# Evaluar el modelo en el conjunto de entrenamiento
accuracy_train = logistic_model.score(X_train_vectorized, y_train)
print(f'Accuracy en el conjunto de entrenamiento: {accuracy_train:.2f}')




c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy en el conjunto de prueba: 0.85
Accuracy en el conjunto de entrenamiento: 0.86


In [86]:
# # Extraer los coeficientes del modelo y convertirlos en valores absolutos
# coefficients = np.abs(logistic_model.coef_).flatten()
# vocabulario = tfidf_vectorizer.get_feature_names_out()
# # Crear un DataFrame con los términos y sus coeficientes absolutos
# coef_df = pd.DataFrame({
#     'unigram': vocabulario,
#     'AbsCoefficient': coefficients
# })

In [89]:
# # Calcular el percentil 95 de los valores absolutos de los pesos
# threshold = np.percentile(np.abs(coefficients), 97.5)
# threshold
# # Filtrar unigrams cuyo peso sea mayor que el percentil 95 en valor absoluto
# significant_unigrams = coef_df[np.abs(coef_df['AbsCoefficient']) >= threshold]
# print(significant_unigrams)


                 unigram  AbsCoefficient
61                abiert        1.322308
85                 abord        1.304582
104         abortaresmat        1.670981
129        abortoalegaly        1.692540
133            abortocer        1.796418
...                  ...             ...
19879             vulner        1.572934
20064            yonopar        1.757894
20092  yovotoabortolegal        1.868827
20099              yovoy        2.380295
20176               zurd        2.273082

[506 rows x 2 columns]


In [95]:
# top_features = coef_df.sort_values(by='AbsCoefficient', ascending=False)

# # Obtener las 10 características con mayor peso
# top_features_reg = top_features.head(506)

# print("Las 20 características más importantes:")
# print(top_features.head(10))

Las 20 características más importantes:
                    unigram  AbsCoefficient
13147  noalabortoenargentin       10.486097
239            abortolegaly        9.005599
1609        argentinaesprov        5.916564
181             abortolegal        4.327035
13141             noalabort        4.232289
12197           mentiraverd        4.004185
15760     queelabortosealey        4.003870
4777        cuidemoslasdosv        3.965239
291           abortoseraley        3.936765
15824             quesealey        3.916897


In [96]:
# unigram_df = pd.DataFrame(top_features_reg[['unigram', 'AbsCoefficient']])
# # Extraer solo las palabras de top_20_features
# top_features_words = unigram_df['unigram'].tolist()
# # Crear un nuevo vectorizador usando solo las palabras de top_20_features
# custom_vectorizer = TfidfVectorizer(vocabulary=top_features_words)

# # Ajustar y transformar el DataFrame original
# X_custom = custom_vectorizer.fit_transform(df_preproces['cleaned_tweet'])

# # Convertir la matriz resultante a un DataFrame
# X_custom_df = pd.DataFrame(X_custom.toarray(), columns=custom_vectorizer.get_feature_names_out(), index=df_preproces.index)

# # Agregar las nuevas características al DataFrame original
# df_unigramas = pd.concat([df_preproces, X_custom_df], axis=1)

# # Mostrar el DataFrame original con las nuevas columnas
# print("\nDataFrame Original con Unigramas de Mayor Peso:")
# df_unigramas.columns


DataFrame Original con Unigramas de Mayor Peso:


Index(['cleaned_tweet', 'postura', 'noalabortoenargentin', 'abortolegaly',
       'argentinaesprov', 'abortolegal', 'noalabort', 'mentiraverd',
       'queelabortosealey', 'cuidemoslasdosv',
       ...
       'recuper', 'prev', 'fren', 'decid', 'jubil', 'austin', 'hijx',
       'multipl', 'pav', 'critic'],
      dtype='object', length=508)

In [92]:
# comunidades_unigramas=df_unigramas.drop(['cleaned_tweet'], axis=1)

In [93]:
# comunidades_unigramas.to_csv('G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/comunidades_unigramas.csv')

Realizamos un análisis Chi-cuadrado para evaluar la relación entre los unigrams (palabras) y la variable objetivo en un conjunto de datos.  Filtramoa los unigrams con p-valores significativos (menores que 0.05) para identificar los unigramAs más relevantes.

In [18]:
# Calcular Chi-cuadrado
chi2_values, p_values = chi2(X_train_vectorized, y_train)
vocabulario = tfidf_vectorizer.get_feature_names_out()

# Crear un DataFrame para los resultados de Chi-cuadrado
chi2_results = pd.DataFrame({
    'unigram': vocabulario,
    'chi2': chi2_values,
    'p_value': p_values
})

# Filtrar unigramas con p-valor significativo (ejemplo: p < 0.05)
chi2_results = chi2_results[chi2_results['p_value'] < 0.05]

# Ordenar por valor Chi-cuadrado
chi2_results = chi2_results.sort_values(by='chi2', ascending=False)

# Imprimir los resultados de Chi-cuadrado
print("\nTop Unigramas por Chi-cuadrado:")
print(chi2_results.head(10))


Top Unigramas por Chi-cuadrado:
                    unigram         chi2        p_value
13147  noalabortoenargentin  2345.370055   0.000000e+00
239            abortolegaly  1527.739043   0.000000e+00
1609        argentinaesprov  1492.924898   0.000000e+00
13141             noalabort   898.163564  2.460595e-197
17436                sialav   795.591805  4.903265e-175
16963       salvemoslasdosv   759.921154  2.793858e-167
19565                   vid   677.522080  2.310943e-149
15824             quesealey   609.594095  1.370990e-134
15760     queelabortosealey   418.746851   4.573091e-93
181             abortolegal   372.228658   6.122799e-83


# Agregamos los unigramas más significativos como columnas

In [19]:

# Extraer solo las palabras de top_20_features
top_features_words_chi = chi2_results['unigram'].tolist()
# Crear un nuevo vectorizador usando solo las palabras de top_20_features
custom_vectorizer = TfidfVectorizer(vocabulary=top_features_words_chi)

# Ajustar y transformar el DataFrame original
X_custom = custom_vectorizer.fit_transform(df_preproces['cleaned_tweet'])

# Convertir la matriz resultante a un DataFrame
X_custom_df = pd.DataFrame(X_custom.toarray(), columns=custom_vectorizer.get_feature_names_out(), index=df_preproces.index)

# Agregar las nuevas características al DataFrame original
df_unigramas_chi = pd.concat([df_preproces, X_custom_df], axis=1)

# Mostrar el DataFrame original con las nuevas columnas
print("\nDataFrame Original con Unigramas de Mayor Peso:")
df_unigramas_chi.columns


DataFrame Original con Unigramas de Mayor Peso:


Index(['cleaned_tweet', 'postura', 'noalabortoenargentin', 'abortolegaly',
       'argentinaesprov', 'noalabort', 'sialav', 'salvemoslasdosv', 'vid',
       'quesealey',
       ...
       'degener', 'judi', 'almorzandoconml', 'acuerd', 'facultad',
       'vamosargentin', 'estoyverd', 'lousteau', 'virg', 'renov'],
      dtype='object', length=1283)

In [20]:
comunidades_unigramas_chi=df_unigramas_chi.drop(['cleaned_tweet'], axis=1)
comunidades_unigramas_chi.columns

Index(['postura', 'noalabortoenargentin', 'abortolegaly', 'argentinaesprov',
       'noalabort', 'sialav', 'salvemoslasdosv', 'vid', 'quesealey',
       'queelabortosealey',
       ...
       'degener', 'judi', 'almorzandoconml', 'acuerd', 'facultad',
       'vamosargentin', 'estoyverd', 'lousteau', 'virg', 'renov'],
      dtype='object', length=1282)

In [ ]:
comunidades_unigramas_chi.to_csv('G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/Comunidades_unigramas_chi.csv', index=False)

# ANALISIS DE SENTIMIENTOS

In [21]:
comunidades_filtradas_filtrada['full_text_cleaned']

0         Contamos con una ventaja que ellos no tienen: ...
1         En el fondo lo que les jode es nuestra liberta...
2         A vos que decís #Salvemoslasdosvidas te pido q...
3         Diputadxs: ¿Qué significa para ustedes una muj...
4         Que no coincidamos en el tema del aborto con  ...
                                ...                        
503323                     Y una vida terminada #NoAlAborto
503328    Ojalá los médicos se organicen y marchen para ...
503333    Debe estar senil ya pobre. Avísenle que el bla...
503334    La Academia Nacional de Derecho advierte que l...
503339    Gastón no pierdas el tiempo con estas personas...
Name: full_text_cleaned, Length: 110409, dtype: object

In [22]:
import pandas as pd
import re

# Cargar el diccionario NRC en español como un archivo CSV con columnas 'palabra' y 'sentimiento'
diccionario = pd.read_csv("G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/lexico_nrc.csv")

# Convertimos a conjuntos de palabras para agilizar la búsqueda
palabras_positivas = set(diccionario[diccionario['sentimiento'] == 'positivo']['palabra'])
palabras_negativas = set(diccionario[diccionario['sentimiento'] == 'negativo']['palabra'])

# Función para preprocesar el texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'http\S+', '', texto)  # Eliminar enlaces
    texto = re.sub(r'@\w+', '', texto)  # Eliminar menciones
    texto = re.sub(r'#\w+', '', texto)  # Eliminar hashtags
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto)  # Eliminar caracteres especiales
    return texto.split()

# Función para contar palabras positivas y negativas en un tweet
def contar_palabras_sentimiento(tweet):
    palabras = limpiar_texto(tweet)
    positivos = sum(1 for palabra in palabras if palabra in palabras_positivas)
    negativos = sum(1 for palabra in palabras if palabra in palabras_negativas)
    return pd.Series([positivos, negativos])

# Aplicar la función a cada tweet y crear las nuevas columnas en el DataFrame
comunidades_filtradas_filtrada[['cantidad_positivas', 'cantidad_negativas']] = comunidades_filtradas['full_text_cleaned'].apply(contar_palabras_sentimiento)

# Mostrar el DataFrame para verificar los resultados
print(comunidades_filtradas_filtrada[['full_text_cleaned', 'cantidad_positivas', 'cantidad_negativas']].head())


                                   full_text_cleaned  cantidad_positivas  \
0  Contamos con una ventaja que ellos no tienen: ...                   2   
1  En el fondo lo que les jode es nuestra liberta...                   1   
2  A vos que decís #Salvemoslasdosvidas te pido q...                   0   
3  Diputadxs: ¿Qué significa para ustedes una muj...                   0   
4  Que no coincidamos en el tema del aborto con  ...                   2   

   cantidad_negativas  
0                   4  
1                   1  
2                   1  
3                   0  
4                   4  


In [23]:


promedio_por_postura = comunidades_filtradas_filtrada.groupby('postura')[['cantidad_positivas', 'cantidad_negativas']].mean()
print(promedio_por_postura)


         cantidad_positivas  cantidad_negativas
postura                                        
0                  0.862311            1.444756
1                  0.812355            1.211265


In [24]:
comunidades_filtradas_filtrada['razon_positiva_negativa'] = comunidades_filtradas_filtrada['cantidad_positivas'] / (comunidades_filtradas_filtrada['cantidad_negativas'] + 1)
razon_por_postura = comunidades_filtradas_filtrada.groupby('postura')['razon_positiva_negativa'].mean()
print(razon_por_postura)


postura
0    0.417870
1    0.432239
Name: razon_positiva_negativa, dtype: float64


Análisis de varianza (ANOVA) para comparar las medias de diferentes grupos de datos. Divide el conjunto de datos en dos categorías (positivas y negativas) basándose en la columna 'postura', y luego calcula la ANOVA por separado para las variables 'cantidad_positivas' y 'cantidad_negativas'.

In [25]:
from scipy.stats import shapiro

for nombre, grupo in comunidades_filtradas_filtrada.groupby('postura'):
    print(f"Postura {nombre}, Shapiro-Wilk cantidad_positivas:", shapiro(grupo['cantidad_positivas']))
    print(f"Postura {nombre}, Shapiro-Wilk cantidad_negativas:", shapiro(grupo['cantidad_negativas']))


Postura 0, Shapiro-Wilk cantidad_positivas: ShapiroResult(statistic=0.772774930261004, pvalue=3.063687130552448e-116)
Postura 0, Shapiro-Wilk cantidad_negativas: ShapiroResult(statistic=0.8295502634391125, pvalue=1.888844004391049e-108)
Postura 1, Shapiro-Wilk cantidad_positivas: ShapiroResult(statistic=0.7708252321498745, pvalue=1.2754359693626166e-125)
Postura 1, Shapiro-Wilk cantidad_negativas: ShapiroResult(statistic=0.7853933898557103, pvalue=9.853184021042165e-124)


c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 45254.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 65155.
  res = hypotest_fun_out(*samples, **kwds)


In [26]:
from scipy.stats import levene, bartlett
grupos_positivos = [grupo['cantidad_positivas'].values for nombre, grupo in comunidades_filtradas_filtrada.groupby('postura')]
grupos_negativos = [grupo['cantidad_negativas'].values for nombre, grupo in comunidades_filtradas_filtrada.groupby('postura')]
# Levene test (recomendado si los datos no son perfectamente normales)
print("Levene cantidad_positivas:", levene(*grupos_positivos))
print("Levene cantidad_negativas:", levene(*grupos_negativos))

# Bartlett test (para datos normales)
print("Bartlett cantidad_positivas:", bartlett(*grupos_positivos))
print("Bartlett cantidad_negativas:", bartlett(*grupos_negativos))


Levene cantidad_positivas: LeveneResult(statistic=51.417690108217165, pvalue=7.512822050599579e-13)
Levene cantidad_negativas: LeveneResult(statistic=306.1445208424068, pvalue=1.869519532618154e-68)
Bartlett cantidad_positivas: BartlettResult(statistic=203.20034952558615, pvalue=4.182837396975247e-46)
Bartlett cantidad_negativas: BartlettResult(statistic=250.95846586979462, pvalue=1.6050443807963364e-56)


In [27]:
from scipy.stats import f_oneway

grupos_positivos = [grupo['cantidad_positivas'].values for nombre, grupo in comunidades_filtradas_filtrada.groupby('postura')]
grupos_negativos = [grupo['cantidad_negativas'].values for nombre, grupo in comunidades_filtradas_filtrada.groupby('postura')]

anova_positivos = f_oneway(*grupos_positivos)
anova_negativos = f_oneway(*grupos_negativos)

print("ANOVA para palabras positivas:", anova_positivos)
print("ANOVA para palabras negativas:", anova_negativos)


ANOVA para palabras positivas: F_onewayResult(statistic=64.013460907504, pvalue=1.2475938261619496e-15)
ANOVA para palabras negativas: F_onewayResult(statistic=647.6668006348247, pvalue=1.8565518966075216e-142)


In [28]:
sentimientos_comunidad=comunidades_filtradas_filtrada[[ 'cantidad_positivas',
       'cantidad_negativas', 'razon_positiva_negativa', 'postura']]
sentimientos_comunidad

,cantidad_positivas,cantidad_negativas,razon_positiva_negativa,postura
0,2,4,0.400000,1
1,1,1,0.500000,1
2,0,1,0.000000,1
3,0,0,0.000000,1
4,2,4,0.400000,0
...,...,...,...,...
503323,0,0,0.000000,0
503328,0,2,0.000000,0
503333,2,2,0.666667,0
503334,1,2,0.333333,0


In [ ]:
sentimientos_comunidad.to_csv('G:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/Analisis_sentimiento_comunidades.csv')

# Unimos Todas las características 

In [29]:
comunidades_filtradas_filtrada=comunidades_filtradas_filtrada[['postura', 'retweet_count',
       'Question Mark', 'Question Mark Count', 'Starts with Question Mark',
       'Ends with Mark Count', 'Upper Ratio', 'Exclamation Mark',
       'Exclamation Mark Count', 'Ellipsis Occurrence', 'Hashtag Presence',
       'Text Length', 'URL Count', 'Quotes',
       'cantidad_positivas', 'cantidad_negativas', 'razon_positiva_negativa']]
#df_unigramas_filtrado = df_unigramas.drop(columns=['cleaned_tweet', 'postura'])
df_unigramas_chi_filtrado = df_unigramas_chi.drop(columns=['cleaned_tweet', 'postura'])

In [ ]:
# import pandas as pd

# # Supongamos que 'comunidades_filtradas' y 'df_unigramas' son tus DataFrames
# # Asegúrate de que tengan el mismo número de filas
# if comunidades_filtradas_filtrada.shape[0] == df_unigramas_filtrado.shape[0]:
#     # Concatenar DataFrames
#     df_combined = pd.concat([comunidades_filtradas_filtrada, df_unigramas_filtrado], axis=1)
# else:
#     print("Los DataFrames no tienen el mismo número de filas.")

# # Verifica la forma del DataFrame resultante
# print(df_combined.shape)


(110409, 523)


In [30]:
import pandas as pd

# Supongamos que 'comunidades_filtradas' y 'df_unigramas' son tus DataFrames
# Asegúrate de que tengan el mismo número de filas
if comunidades_filtradas_filtrada.shape[0] == df_unigramas_chi_filtrado.shape[0]:
    # Concatenar DataFrames
    df_combined_unigramas_chi = pd.concat([comunidades_filtradas_filtrada, df_unigramas_chi_filtrado], axis=1)
else:
    print("Los DataFrames no tienen el mismo número de filas.")

# Verifica la forma del DataFrame resultante
print(df_combined_unigramas_chi.shape)


(110409, 1298)


In [ ]:
# df_combined.to_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/comunidades_modelos.csv', index=False)

In [ ]:
df_combined_unigramas_chi.to_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/comunidades_modelos_chi.csv', index=False)